# Experiment 201 - A practical guide to experimental uncertainties

**Double click and enter your name and university ID here:** Firstname Lastname 123456789

# A practical guide to experimental uncertainties

Real experiments involve <span class='myclass'>real</span> measurements, which are not perfect. Because of parasitic phenomena, sensitivity of the experiment to noise, or mis-reading of scales, the results we get are generally not the exact values we would like them to be. Moreover, when repeating the same measurement, we often obtain a different result! These "errors" in measurements, more properly called **uncertainties**, need to be properly estimated and quoted when you write up your results. Failure to do so may mean that your results are useless. Consider an error of 1 mm on a distance measurement. If you are measuring the distance to a planet, a 1-mm error is actually very small and that measurement very accurate. But if you happen to try to determine the length of a molecule (which are typically much smaller than 1 mm), then a 1-mm error is pretty bad, and it really means that you have no idea about the length of that molecule. It all depends on context! 

This tutorial/assignment explains you **first** how to estimate uncertainties in individual and repeated measurements  (part A) and **second** how to "propagate" these in calculations (part B).

It mixes material to read with practial exercises to run with Python. **These exercises are directly relevant to what you will have to do in the lab.** You will probably find yourself re-use some of the code below to calculate results of *other* experiments. So the more seriously you do this assignment, the easier your life will be in the lab.

## A. Estimating uncertainties in measurements

Here you are, facing the measuring apparatus of your experiment. This could be a simple ruling stick, a dial with a moving pointer, or the digital display of some sensor. You are reading a value off that measuring tool. What is the uncertainty in that value?

### A.1 Uncertainty in an individual measurement

The uncertainty in a <i>non-repeated</i> measurement can be estimated as **the minimum resolution of the measuring device**. This is often quoted as being half of the smallest division of the measuring apparatus. For example, a reasonable estimate of uncertainty in a single measurement with a typical meter stick, which has ticks every one millimeter, is &plusmn;&nbsp;0.5&nbsp;mm. Similary, with a digital voltmeter that diplays three significant digits, like 3.73&nbsp;V, the uncertainty would correspond to half the value associated with the least significant digit, in this case &plusmn;&nbsp;0.005&nbsp;V.

**It may happen that the value you are trying to read is slowly, randomly varying over time,** because of parastic phenomena (e.g. vibrations). Say for example that your voltage reading varies betwen 3.68 and&nbsp;3.76&nbsp;V. In that case, you would quote the measurement as the mean value between the minimum and maximum value you can read, (3.68 V + 3.76 V) / 2 = 3.72 V, and the uncertainty as half the spread of values, (3.76&nbsp;V&nbsp;&minus;&nbsp;3.68 V) / 2 = 0.04 V, giving you for this example a reading of 3.72&nbsp;&plusmn;&nbsp;0.04&nbsp;V. More on this below.

As a general rule, if you estimate the uncertainty of a quantity in two different ways, and you get two different results (e.g. &plusmn;&nbsp;0.005&nbsp;V from the resolution, and &plusmn;&nbsp;0.04&nbsp;V from the fluctuations), you use the **largest** value for the uncertainty. Essentially, you neglect the one that has the least influence.

Now consider again the above example, but this time imagine the oscillations are slow enough that you can read ten different values of that measurement. You put these in a Python/NumPy array (you will learn more about that if you do the \#202 or \#213 experiment on Python):

In [1]:
import numpy as np
values = np.array([3.68, 3.78, 3.77, 3.71, 3.66, 3.70, 3.71, 3.73, 3.71, 3.74])
print(values)

[3.68 3.78 3.77 3.71 3.66 3.7  3.71 3.73 3.71 3.74]


What is the uncertainty in each of the *individual* measurements above? Clearly, it is **NOT** half the smallest digit, &plusmn;&nbsp;0.005&nbsp;V, as it is obviously varying more than that. Here, **the standard deviation of your data** can serve as the uncertainty of individual measurements. The rationale is that, for process fluctuating based on Gaussian statistics, 68% of repeated measurements occur in a range of the mean value &plusmn;&nbsp;1&nbsp;standard deviation. Therefore, the uncertainty in one additional measurement is **approximately** (see next Section for some extra details) one standard deviation of the previous measurements.

<font color='red'>Calculate the standard deviation of the values above using Numpy's `np.std(...)` function.</font> Type your code for this operation in the empty cell below. Make sure to execute it by typing `Shift + Enter` within the code cell, or by clicking the play button next to the cell (in Colab) or in some toolbars at the top (Jupyter). **NOTE:** You first need to execute the cell above (with `Shift + Enter`) to load Numpy and define the `values` variable.

In [5]:
np.std(values)

0.03534119409414452

Python's answer to the standard deviation has many decimals places. Given that the data you started with, in the `values` vector above, have only two decimal places, it does not make any sense to include more than two decimal places in the uncertainty -- because you do not even know the data that well. **In fact, there is never a good reason to quote an uncertainty with more than one or two significant figures.** The rationale is the following: If a voltage is known as 5.26&nbsp;&pm;&nbsp;0.03251762&nbsp;V, it means that you don't know the 2nd decimal place better than &pm;&nbsp;0.03&nbsp;V, and that the real voltage can be anywhere between 5.23&nbsp;V and 5.29&nbsp;V (and even outside that range 32% of the time). Clearly that means that you have no idea of the values of any subsequent decimal places, and there is no point in quoting it.

<font color='red'>Take this into account to type below the uncertainty you have obtained.</font>

### A.2 Uncertainty in a mean
In the previous section, you calculated the standard deviation of the individual measurements.
Now consider that, in the hope of getting a "better" result than the individual measurements, you take the *mean* of the ten measurements contained in the `values` vector defined above. What is the uncertainty in the mean?

Owing to the mean being centered between lower and higher values, that uncertainty is expected to be **less** than the uncertainty of an individual measurement. This suggests that the standard deviation of the array of values is **NOT** an estimate of the uncertainty in the mean.

The uncertainty in a mean is also expected to decrease with additional measurements (as the mean converges to its 'true' value). The fact that the standard deviation may increase or decrease with additional measurements suggests again that the standard deviation is **NOT** an estimate of the uncertainty in a mean.

The correct way of estimating the uncertainty in a mean is to **divide the standard deviation of the values by the square root of the number *N* of values**. This results in an uncertainty that is less than the standard deviation and decreases with increasing *N*, both requirements for an uncertainty in a mean. This is called the **standard error of the mean** or simply **standard error**, often labelled as $\sigma_\mathrm{m}$. If $\sigma$ denotes the standard deviation of the data, we thus have

$$\sigma_\mathrm{m} = \frac{\sigma}{\sqrt{N}}\,.$$

An additional problem occurs when the amount of data collected (the "sample" of size $N$) is small. In that case, the standard deviation of the **sample**, $s$ (what you have obtained above with the `np.std(..)` function), is not quite the standard deviation of the underlying probabilitic **population**, $\sigma$. In essence, you do not know the value of $\sigma$ in the above equation. To estimate the latter, one needs **to scale the sample standard deviation $s$ , as**

$$\sigma \approx \sqrt{\frac{N}{N-1}} s\,.$$

Note how $\sigma$ approaches the sample standard deviation $s$ for a large number of samples, $N$. The above expression is actually the one you want to use to calculate uncertainties in individual measurements. With that, the standard error becomes

$$\sigma_\mathrm{m} = \frac{s}{\sqrt{N-1}}$$

**So, to quote the result of an experiment in which your measure the same quantity *N* times, you would quote the mean of your samples as the nominal value, and the standard error $\sigma_\mathrm{m}$ as its uncertainty.**

<font color='red'>Calculate the mean of the `values` vector (using Numpy's `np.mean(...)` function). Assign it to a variable and print it.</font>

In [ ]:
values_mean = ...
print(values_mean)

<font color='red'>Now calculate the standard error on that mean value.</font> For that use the `np.std(..)` function as you have done above, the syntax `values.size` to get the number of elements *N* in the `values` array, and the `sqrt(...)` function (from the **math** module) to calculate square roots.

We earlier discussed how many decimal places would be significant to report. Let us show you how to format your answers to display your answers with the appropriate amount of decimal places. For example, let us present the mean of `values` to two decimal places, just like the individual measurements:

In [26]:
print(f'The mean value with two decimal places is {values_mean:.2f}.')

The mean value with two decimal places is 3.72.


With this example, display the standard deviation of the mean with the correct amount of decimal places:

### A.3 Random processes

Sometimes, the value you measure varies from realization to realization, because it comes from an intrinsically random process. For example in experiment&nbsp;\#251, *Radioactivity*, the number of radioactive decays counted in a set time interval varies from measurements to measurements. *This variation is caused by the physics of radioactive decay, not by any imprecision in your measurement*. Here estimating the uncertaintly (which really is the standard deviation that you would observe over many such measurements) should be guided by your knowledge of the random process. For radioactive decay counts, which follow a Poisson distribution, a count of *N* is affected by an uncertaintly of $\sqrt{N}$.

**So, in this case, if you detect *N* events in a count, the nominal value for the number of events would be *N*, and the uncertainty on that number of events would be the square-root *N*. Hence you would quote the number of events (in a given time interval) as being $N \pm \sqrt{N}$. Do not use the concept of standard error for this case.**

### A.4 Relative uncertainties

When you quote uncertainties in the form 3.1&nbsp;&plusmn;&nbsp;0.2, we say that&nbsp;0.2 is the <b>absolute</b> uncertainty. 

You can also use <b>relative uncertainties</b>, i.e. quote uncertainties using percentages. In this way, you quote the uncertainty as a percentage of the nominal value. For the example above, 0.2 / 3.1 x 100% = 6.5%, so we could quote it as 3.1&nbsp;&plusmn;&nbsp;6.5%. 

<b>Relative uncertainties help you put things into perspective.</b> If we are measuring a distance, you may think that 100&nbsp;m is a "big" uncertainty, but without knowing the magnitude of the actual distance that statement ("big" uncertainty) does not make any sense. 

If we were measuring the distance between a home and the nearest dairy, about 50&nbsp;m away, then a 100&nbsp;m uncertainty is "big". The only thing clear about a distance of 50&nbsp;m&nbsp;&plusmn;&nbsp;100&nbsp;m is that, really, we do not know that distance without any meaningful precision. And indeed the relative uncertainty on that measurement is 100/50 x 100% = 200%.

Contrast that with the measurement of the distance between the top of two of Auckland's volcanic cones, say 6.5&nbsp;km away. Now the distance is 6500&nbsp;m&nbsp;&plusmn;&nbsp;100&nbsp;m =&nbsp;6500&nbsp;m&nbsp;&plusmn;&nbsp;1.5%. Now that uncertainty of 100&nbsp;m is not so big after all. The measurement is precise within&nbsp;1.5%.

<font color='red'>Calculate the **relative** uncertainty of 175&nbsp;&plusmn;&nbsp;12.</font> Express you result in percentage (%).  
**NOTE**: you can use Python as a normal calculator. Just type, e.g. `(1+5)*2`

<font color='red'>Use the formatting of real numbers from earlier in the assignment to present the relative uncertainty in a "meaningful" number of significant digits.</font>

<font color='red'>Calculate the **absolute** uncertainty of 1285&nbsp;&plusmn;&nbsp;3.5%.</font>

<font color='red'>Again, present the above result using a "meaningful" number of significant digits.</font>

Consider a radio-active decay process (see Section A.3 above) where a count of events is $N \pm \sqrt{N}$. The relative error on that count (uncertainty divided by nominal value) is given by $\sqrt{N}/N = 1/\sqrt{N}$. Observe how that **relative** uncertainty decreases as *N* increases. So when counting events, you get a better accuracy by counting more events, which is what you would expect intuitively. <font color='red'>How many events should you wait for your detector to count to have a 5% accuracy on your count? Calculate that in the cell below.</color>

**Note:** In Python, the power operator is `**`, like 4\**2&nbsp;= &nbsp;4<sup>2</sup>.

## B. Propagation of uncertainties

Ok. We now have measured some data AND have estimated the uncertainties in these data. In a typical experiment, we would now use these data to calculate some results. The question is, given the uncertainties in the data, what is the uncertainty in the result. This procedure is called *propagation of uncertainties* and is an important part of the job. 

Imagine you are traveling a distance of 20&nbsp;m in 4&nbsp;s. Your velocity is 20&nbsp;m&nbsp;/&nbsp;4&nbsp;s =&nbsp;5&nbsp;m/s. Now if that distance is known &plusmn;&nbsp;2&nbsp;m, and the time &plusmn;&nbsp;1&nbsp;s, what is the uncertainty on the velocity? It is easy to think through this by considering what could be the minimal velocity (smallest distance over largest possible time, (20&minus;2)&nbsp;m&nbsp;/&nbsp;(4+1)&nbsp;s =&nbsp;3.6&nbsp;m/s) as well as maximal velocity (largest distance over smallest possible time, (20+2)&nbsp;m&nbsp;/&nbsp;(4&minus;1)&nbsp;s =&nbsp;7.3&nbsp;m/s). Then you take the mean and (half) the spread between these two values to arrive at a result of 5.45 &plusmn; 1.85 m/s.

Clearly, you can see that this will quickly become very tedious, especially for complex mathematical formulae, and there must be a better way. Below we will see the (simplified) rules that govern the *arithmetics of uncertainties*, and how to deal with that efficiently using Python.

### B.1 How to represent uncertainties in Python

Python provides the **uncertainties** module to make calculations with uncertainties easy. The basic module is loaded with the statement `from uncertainties import *` (which in Python means, load all (\*) the functions of the uncertainties module). You will see that statement in the cell below, in which we also load the standard **math** module to have access to other math functions, in particular the square-root function, `sqrt`. <font color='red'>Execute the cell below to get you started.</font>

**NOTE:** The `uncertainties` module is available by default on the Advanced Lab computers, but not in Colab, and if using you own Python installation it may also be missing. The `from ...` command below would then return an error. If using Colab, execute the line below (starting with `!pip ...`) to download and install it in your environment (this may take a few seconds to complete). If using your own install, check how to run the `pip` command in the documentation. Alternatively, you can dowload the package from https://pypi.python.org/pypi/uncertainties. 

In [37]:
!pip install uncertainties   # Run this line to install the module if you haven't it yet (ie, if executing the cell below trigger an error)
from uncertainties import *

After having loaded the uncertainties module, a new type called <b>ufloat</b> is available. It allows to associate an error (standard deviation) to any normal float number. <font color='red'>Execute the three cells below to see what they do.</font>

In [38]:
x = ufloat(4, 0.1)      # this means  4 +/- 0.1
y = ufloat(10, 0.3)     # this means 10 +/- 0.3

In [39]:
print(x)     # printing x should now display the uncertainty, as 4.0+/-0.1

4.00+/-0.10


In [40]:
y            # no need for print(...) if one simply wants to display the value of a single variable

10.0+/-0.3

#### Basic calculations and syntax

One can then perfom calculations with uncertainties using normal Python operators. <font color='red'>Execute the two cells below to see it in action.</font> In principle, that is (almost) all there is to know, but keep reading. 

In [41]:
x**2

16.0+/-0.8

In [42]:
a = x + y
print(a)

14.00+/-0.32


You can access the nominal value and uncertainty using the dot notation as below. This will come handy later. Suppose x = 4.0 &plusmn; 0.1. Then

> `x.nominal_value` (or `x.n` as shortcut) returns 4.0  
> `x.std_dev` (or `x.s` as shortcut) returns 0.1  
> `x.s/x.n*100` returns the relative uncertainty in %, here 2.5%

***

<font color='red'>Try these below.</font>

The **uncertainties** module also works on Numpy arrays of numbers, facilitating parallel calculations over many values. Check the documentation (available on the web) for these cases.

### B.2 Basic rules for addition/substraction/multiplication/division

Rather than blindly accepting what Python puts out, let us try to understand how uncertainties are propagated in a calculation. Assuming that all our uncertainties have a *Gaussian statistical distribution*. For x = 4 &plusmn; 0.1, that means that the probability distribution that leads to these (seemingly) random values for *x* as we keep measuring it have a Gaussian distribution, centered on 4 (the mean) and with a standard deviation of 0.1. In a Gaussian distribution there is a 68% chance that the actual (exact) value is within &plusmn;&nbsp;1 standard deviation of the nominal value (here, 4). Distributions are not always Gaussians but there are many reasons why they often turn out to be Gaussian. In particular, this is often the case for *repeated measurements* where you have estimated the uncertainties with the standard deviation and standard error (see part A).

When **adding or subtracting** two values with Gaussian uncertainties, $z = x + y$ or $z = x - y$, the laws of probabilites predict that the uncertainty $\sigma_z$ on the result $z$ is given by
$$\sigma_z = \sqrt{\sigma_x^2 + \sigma_y^2} \qquad \text{when adding or substracting $x$ and $y$}$$
where $\sigma_x$ ($\sigma_y$) is the uncertainty of $x$ ($y$). If you have more than two variables to add, just keep adding the uncertainties-squared inside the square-root in the above formula. This way of "adding" the (absolute) uncertainties by "taking the square-root of the sum of the squares" is said to be a **quadrature sum of uncertainties**. The fact that the uncertainty $\sigma_z$ is not just the direct sum of the uncertainties (which would be larger) reflects the fact that, given that the uncertainties are random (and have random signs), they will not always act in the same direction. And when you take into account all the possibilities, with their probabilities, you get the quadrature-sum result. Notice also how uncertainties always **add**, even when **substracting** the values.

When **multiplying or dividing**  two values with Gaussian uncertainties, $z = x * y$ or $z = x / y$, the **relative**  uncertainty $\sigma_z/z$ on the result is the quadrature sum of the **relative** uncertainties,
$$\frac{\sigma_z}{z} = \sqrt{\left(\frac{\sigma_x}{x}\right)^2 + \left(\frac{\sigma_y}{y}\right)^2} \qquad \text{when multiplying/dividing $x$ and $y$}$$
So, the rule is in the essence the same as for addition and substraction, except that we have to use the **relative** uncertainties instead of the **absolute** ones. Again, this can be generalized to any number of variables.

<font color='red'>Verify the propagation formula for addition above by applying it to add x = 4 &plusmn; 0.1 to y = 3 &plusmn; 0.2. In the cell below, assign these two values to variables, print the sum (`print(x+y)`), which will also show the uncertainty on the result as calculated by Python and the `uncertainties` module, and then calculate **explicity** the same uncertainty using the quadrature-sum rule (don't just type the numbers directly in your calculation; rather access the uncertainty  on `x` with `x.s` and so on &mdash; see above). Print the result and verify that you get the same uncertainty with the two methods. If not, check your code.</font>

**NOTE:** you can have multiple statements, including mutliple print statements, in the same cell, by writing them on top of each other on separate lines. Everything is evaluated at once, and all the results shown below, when you press `Shift+Enter`. Also note that assigning to a variable does not produce any output (like `u = 4 + 5`). You need to add a `print(u)` statement to see the result.

In [ ]:
x = ...        # 4 +/- 0.1
y = ...        # 3 +/- 0.2
print(x+y)
print(...)     # explicit calculation. Use sqrt(...), x.s, and the ** operator for powers.

<font color='red'>Repeat the exercise for substraction, (75 &plusmn; 3) &minus; (71 &plusmn; 4), both directly (the Python way; assign the result to a variable called `z` &mdash; it will come handy below) and explicitely.</font>

In [ ]:
x = ...        # 75 +/- 3
y = ...        # 71 +/- 4
z = ...
print(z)
print(...)     # explicit calculation of uncertainty

<font color='red'>Calculate and print the *relative* uncertainties on $x$ and $y$, as well as that of their difference $x-y$ (which is stored in `z` from the code above). Compare them and comment your observations in the next cell. How does the relative uncertainty on $z=x-y$ compare with that of $x$ and $y$? Why? Explain.</font>

In [ ]:
x_rel = x.s/x.n*100
print(f'x: {x_rel:.2f}%')   # this neatly prints the relative uncertainty of x with 2 decimal places using the 'f' (for 'format') string syntax

print(...)     # do the same for y

print(...)     # do the same for the difference. Call the variable z defined above.

*Comment on the result here*

You sould by now have discovered that substracting two values close from each other, and whose uncertainty intervals overlap (or nearly overlap), gives a very poor estimate of the result. (Try to apply the reasoning mentioned at the start of section B, with the example of the velocity, to this case.) The key message here is that, if the uncertainty on a result you obtain is not  satisfactory (not accurate enough for what you need), you have no choice but to go back to your measurements and attempt to get lower uncertainies on $x$ and&nbsp;$y$ (for example by averaging over a larger set of repeated measurements).

<font color='red'>Consider the *multiplication* z = x\*y. Calculate the uncertainty with Python, and then explicitely with the multiplication rule. Check the results are the same.</font> Keep in mind that the quadrature-sum rule for multiplication/division returns the **relative** uncertainty of the result (rather than the absolute uncertainty).

<font color='red'>Now consider the following problem. Two forces $F_1 = 45 \pm 3$&nbsp;N and $F_2 = 18 \pm 4$&nbsp;N act on the same object, in the same direction, and moves it by a distance $d = 3 \pm 0.1$&nbsp;m in a time $t = 8.0 \pm 1.5$&nbsp;s. Calculate the **power** used to realize this displacement, $$P = \frac{(F_1 + F_2) \cdot d}{t}\,,$$including its uncertainty, directly using Python, and also explicitely using the quadrature sum rules above. Take as many lines and define as many intermediate variables as necessary &mdash; see if you can get the two uncertainties to match.</font> By this time, you should start to realize that Python's `uncertainties` module really make your life simpler!

In [ ]:
F1 = ...      # 45 +/- 3 N
F2 = ...
d = ...
t = ...
P = ...
print(P)
...           # explicit calculation of uncertainty

### B.3 A few more rules and the concept of independent/dependent variables

<font color='red'>Set x = 5 &plusmn; 0.4 and calculate z = 2x with Python. Print the result neatly &mdash; code provided below.</font>

In [ ]:
x = ...
z = ...

print(f"     x = {x:5f}")      # the '5f' means: format the number as a Float, right justified over 5 spaces
print(f"z = 2x = {z:5f}")

<font color='red'>Now cleverly consider that 2x = x + x and apply (explicitely) the rule for the uncertainty of the sum. What uncertainty do you get for the result this way?</font>

Notice  how your two results are **different**. What is going on? And what is the right way? Also notice that the uncertainty calculated by Python simply appears to be 2 times the uncertainty on `x`. 

The trick here is that **the quadrature-sum rules for the propagation of uncertainties** laid out above (for the sum/difference/product/division) **are only correct if the uncertainties in the different variables are independent**. What does that mean? When we say *x* = 5 &plusmn; 0.4, all we know is that there is a 68% chance that the TRUE value of  and *x* lies in the interval [5 &minus; 0.4 ; 5 + 0.4]. In particular, we do not know if it is larger or smaller than&nbsp;5. If I consider a second variable, *y* = 3 &plusmn; 0.2, again I do not know whether the true value of *y* is larger or smaller than&nbsp;3. Now if *x* and *y* are *independent*, the fact that the true value of *x* is &mdash; say &mdash; larger than&nbsp;5 does not imply anything as to the true value of *y*. It could still be larger or smaller than&nbsp;3 with equal probabilities. There are no **correlations** between the errors on the two variables &mdash; in particular their relative sign is random. In contrast, **dependent** variables would exhibit such correlations. For example, we may know, because of specificities of the experiment, that if I measure a value for *x* that is too large, then my measurement for the value of *y* will also be too large. Such uncertainties are said to **correlated**, hence **not independent**. If the uncertainties of two variables systematically have the opposite sign, they are said to be **anti-correlated**.

Clearly, when I calculate *x* + *x* the two terms in this addition are actually the same, and therefore are **not indepedent**. The quadrature sum rule therefore does not apply, and the 2nd result calculated above is NOT correct. Python gives the right result. In fact, the Python's **uncertainties** module is quite clever. Even if you have many intermediate results, that depend somehow on the same variable(s), it will correctly track the dependant and independant variables for you. This makes your life much (much) easier!

Now you may start to wonder what is the *right* rule to calculate the uncertainty of *z* = 2 *x*? Well, consider that we can write *z* = 2 *x* as z = x \* y, with y = 2. Note that the variable *y*, which is the (exact) number&nbsp;2, has no correlation with *x* whatsoever. These are independent variables and we can apply the product rule for uncertainties as defined above. But what is the uncertainty on *y*? Well clearly, it is zero, $\sigma_y = 0$: becasue of factor of 2 is an **exact** number, it has no uncertainties.

<font color='red'>With that in mind, calculate **explicity** the uncertainty on z = 2*x with the <b>product</b> rule and see how it compares with Python's result above (you should get the same uncertainty as Python).</font>

From your calculation, the following new rule should be clear: **When multiplying/dividing a variable $x$ by an exact scalar $a$ (or vice versa), $z = a x$, or $z = x/a$, or $z = a/x$,  the relative uncertainty on $z$ is the same as the relative uncertainty on $x$:**

$$\frac{\sigma_z}{z} =  \frac{\sigma_x}{x}$$

<font color='red'>Set *x* = 25 &plusmn; 2. Calculate z = 1/*x* (with Python). Then calculate and print the **relative** uncertainties on *x* and *z*. Check that they are the same.</font> **NOTE:** you can copy the percentage-print statement used in Section B.2 above to have a neat output.

In [ ]:
x = ...
z = ...
print(...)
...

We have one more important basic rule to look at. This one applies to powers. If I try to calculate the uncertainty in $z = x^2$ by writing it as $z = x * x$, and by applying the product rule, clearly I will have a problem: the two factors in my product are again **not independent** (they are the same). The rule for this case is the following:

**For $z = x^a$, where $a$ is an exact scalar, the relative uncertainty in $z$ is $a$ times the relative uncertainty of $x$,**

$$\frac{\sigma_z}{z} = a \frac{\sigma_x}{x}$$
In fact, if you think about it carefully, this is like applying the product rule (quadrature sum of relative uncertainties) but instead of doing a **quadrature sum**, you do a **normal sum** instead. The rationale is that, if the variables are dependent, like in $x * x$, we consider the worst case scenario with all uncertainties acting **in the same direction**, rather than in random directions.

<font color='red'>Exercise: consider a uniformly accelerated motion, $x = a t^2/2$, when the acceleration $a = 5 \pm 0.2\ \mathrm{m/s^2}$ and the time $t = 8.3 \pm 0.7\ \mathrm{s}$. Calculate the uncertainty on $x$, directly using Python, and also explicity using all the rules as appropriate.</font>

In [ ]:
a = ...
t = ...
x = ...
print(x)
...                  # explicit calculation of the uncertainty in x

### B.4 Propagation of uncertainties in functions

Sometimes a result is obtained through calculations involving more complex functions than addition and multiplication. To handle this case, one has to load the <b>umath</b> sub-module of the uncertainties module. <font color='red'>Execute the next cell.</font>

In [ ]:
from uncertainties.umath import *

Consider a value $x_1 = 2 \pm 0.1$, and, as an example of a (complicated) function, let us calculate the logarithm of that value with Python. This is done by the code below. <font color='red'>Make sure you understand the code, and execute that cell.</font>

In [ ]:
x1 = ufloat(2, 0.1)
y1 = log(x1)
print(f'x1 = {x1:f}')
print(f'x1: {x1.s/x1.n*100:.2f}%')         # displaying the relative uncertainty
print(f'log(x1) = {y1:f}')
print(f'log(x1): {y1.s/y1.n*100:.2f}%')    # displaying the relative uncertainty

Observe how Python still provides you with the uncertainty on the result. To try to understand the rule behind it, let us try a different value of $x$ but with **the same (absolute) uncertainty**. <font color='red'>Execute the code below.</font>

In [ ]:
x2 = ufloat(15, 0.1)
y2 = log(x2)
print(f'x2 = {x2:f}')
print(f'x2: {x2.s/x2.n*100:.2f}%')         # displaying the relative uncertainty
print(f'log(x2) = {y2:f}')
print(f'log(x2): {y2.s/y2.n*100:.2f}%')    # displaying the relative uncertainty

Comparing the two results does not really make clear what is going on. In the first case, the relative uncertainy of $\log(x_1)$ was larger than that of $x_1$, in the second case it is the other way around. There is no clear pattern in the absolute uncertainties either.

To clarify this point, let us make a plot of the log function. In the plot below, we also highlight (in green) the ranges covered by $x_1 \pm \sigma_{x_1}$ (from $x_1 - \sigma_{x_1}$ to $x_1 + \sigma_{x_1}$) and $x_2 \pm \sigma_{x_2}$. Similarly, we highlight the ranges covered by $y_1$ and $y_2$ (in red), i.e. the logs of $x_1$ and $x_2$, with their  uncertainties as calculated by Python above. This is done using the **matplotlib** module. <font color='red'>You do not need to understand all the details. Just execute the code below, a graph should appear, and make sure you understand what the graph represents.</font>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

xx = np.arange(0.5,20,0.01)   # a range of values, from 0.5 to 20, at 0.01 interval
yy = np.log(xx)               # the log of all these values 

fig = plt.figure(num=1, figsize=(8,6))
plt.fill_between(xx, yy, -1.0, where=(np.abs(xx-x1.n)<x1.s), color='g')
plt.fill_between(xx, yy, -1.0, where=(np.abs(xx-x2.n)<x2.s), color='g')
plt.fill_betweenx(yy, xx, where=(np.abs(yy-y1.n)<y1.s), color='r')
plt.fill_betweenx(yy, xx, where=(np.abs(yy-y2.n)<y2.s), color='r')
plt.title("y = log(x)"); plt.xlabel("x"); plt.ylabel("y")
plt.plot(xx, yy)

Observe first how the two green bars have the same thickness. This is because we have chosen the same **absolute** uncertainties for $x_1$ and $x_2$. However, you can also see how the green bar on the left, centered on $x_1=2$, **intercepts a wider vertical swath of the (blue) log curve** than around $x_2=15$ on the right, as visualized by the different thicknesses of the **red** bars. This naturally explains why the **absolute** uncertainty on $y_1$ is larger than that for $y_2$.

Pushing this to the limit: If the blue curve was purely horizontal (constant function, say $y=2$), the uncertainty on $x$ would have absolutely no influence on the uncertainty on $y$. After all, $y=2$ is an **exact** number, with no uncertainty.

The above discussion should make clear that **what drives the propagation of uncertainties in a function is the slope of that function. A steep slope (larger than&nbsp;1) amplifies absolute uncertainties. A weak slope (lower than &nbsp;1) dampen it.**

This leads to the following equation to calculate propagation of uncertainty through a function. If $y = f(x)$, then

$$ \sigma_y = \frac{df}{dx} \sigma_x $$

<font color='red'>Keeping in mind that the derivative of the $\log(x)$ function is equal to $1/x$, use the above formula to calculate **explicitely** the uncertainty on $y_1 = log(x_1)$. Check that it matches Python's result above.</font>

For a function of several variables, $u = f(x,y,z)$, the result above can be generalized. Not unsurpisingly, one has to do a <b>quadrature sum</b> (assuming uncertainties on $x$, $y$, and $z$ are independent) of the contribution of each variable to the uncertainty. This leads to

$$ \sigma_u = \sqrt{\left(\frac{\partial f}{\partial x} \sigma_x\right)^2 + \left(\frac{\partial f}{\partial y} \sigma_y\right)^2 + \left(\frac{\partial f}{\partial z} \sigma_z\right)^2}$$

*All the specific uncertainty propagation formula described in the previous sections can be derived from this more general result.* 

For example, for the sum rule:

If $u = f(x,y) = x+y$, one has $\partial f/\partial x = 1$ and $\partial f/\partial y = 1$. This leads to $\sigma_u = \sqrt{\sigma_x^2+\sigma_y^2}$ as expected.

<font color='red'>Optional: try to derive the rule for products and powers.</font>

### B.5 A few more exercises

Now use your knowledge of Python and of the **uncertainties** module to solve the problems below. (You do not have to do explicit calculations using quadrature sum here.)

The following are the results of 10 measurements of a length (in mm):

    [32.84, 32.55, 33.10, 33.04, 32.63, 33.02, 32.73, 33.21, 32.95, 32.70]

<font color='red'>(a) Calculate the best estimate of the length (i.e. the mean)</font>

<font color='red'>(b) Calculate the precision of each *individual* measurement?</font>

<font color='red'>(c) How well is the length determined as a result of all ten measurements? I.e. calculate the uncertainty in the mean?</font>

<font color='red'>(d) Quote the measurement of that length in the form xx &plusmn; yy, with an appropriate number of significant figures.</font>

*Type your result here*

<font color='red'>Suppose that $z = x + y$ is the sum of two independent measurements, with $x \simeq50$. Suppose also that $y$ is
approximately one-hundredth of the value of $x$. If the **relative** uncertainty of $x$ is 5%, calculate and print
the **relative** uncertainty in $z$, assuming that the **relative** uncertainty in $y$ is</font>

<font color='red'>(a) &plusmn; 1%,</font>  
<font color='red'>(b) &plusmn; 10%,</font>  
<font color='red'>(c) &plusmn; 30%.</font>

<font color='red'>What conclusion can you draw about estimating the uncertainty in the sum of two quantities when one quantity is much larger than the other?</font>

*Type your answer here*

<font color='red'>Calculate the volume of a cylinder, and its uncertainty, given that it has a height $h$ of 24.7&nbsp;cm&nbsp;&plusmn;&nbsp;0.8&nbsp;cm and a radius $R$ of 8.2&nbsp;mm&nbsp;&plusmn;&nbsp;0.1&nbsp;mm.</font> Remember that the volume of a cylinder is given by $V = \pi R^2 h$. **NOTE:** In Python, you can access the value of $\pi$ with the math or numpy module.

The refractive index of a material determines the ratio of the angle of incidence $\theta_\mathrm{i}$ to the
angle of refraction $\theta_\mathrm{r}$ of light as, starting from air, it crosses the material boundary [Snell's Law $𝑛 = \sin\theta_\mathrm{i}/\sin\theta_\mathrm{r}$]. <font color='red'>If measurements of $\theta_\mathrm{i}$ and $\theta_\mathrm{r}$ have means of $\pi/6$ and $\pi/8$ respectively and the standard error of each is $\pi/200$, what is the best estimate of $n$ and its uncertainty?</font>

***